# Reading the Data

In [51]:
import numpy as np
import scanpy as sc

adata = sc.read_h5ad(
    "/home/jhaberbe/Projects/Personal/TokenGT/data/output-dgi-10-10-20MAY2025.h5ad"
)
adata = adata[adata.layers["transcript"].sum(axis=1) > 20].copy()
adata.obs["log_plin2_area"] = np.log1p(adata.obs["plin2_area"])
adata.obs["log_oil_red_o_area"] = np.log1p(adata.obs["oil_red_o_area"])
adata.obs["log_lipid_droplet_area"] = np.log1p(adata.obs["lipid_droplet_area"])

adata.X = adata.layers["transcript"].copy()
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

/home/jhaberbe/Projects/Personal/TokenGT/.venv/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/jhaberbe/Projects/Personal/TokenGT/.venv/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
adata.X seems to be already log-transformed.


# My word vomit. 

I have a dataset, where I have:

[near_amyloid, log_plin2_area, log_oil_red_o_area, log_lipid_droplet_area]
along with
[size-factors precomputed for each sample]
[log_normalized count data for F features for each sample]
[count data for F features]
[specimen each cell belongs to] 

and I want to have the first part of this model learn to generate embeddings such that we get:
NB feature reconstruction for the counts,
hurdle log_normal distributions for all pathology metrics.
near_amyloid BCE with logits for near_amyloid.
I want this to be trained in a fair fashion, meaning I want adversarial training against the specimen information (think like FairGNN). 

The output at this point should be embeddings generated for all the cells, such that the embeddings fairly distribute specimens across the space.

---

Jointly, we should train a transformer model that takes in the tokens, and computes perplexity via a set of normalizing flows. We will have neighborhood information, such that we can compute 
[center cell, neighbor 1, neighbor 2, ..., neighbor 30]
We will prepend some kind of [cls] vector to the beginning, and we will incrementally train our model to produce hidden states such that the normalizing flow maximizes the entropy (reduces the suprisal) of the next known token. 

I imagine it would involve the following type of procedure:
provide embedding 1, use the transformer and [cls] to generate a normalizing flow to the embedding space for the next token, get the perplexity of the known token, then add the known token and repeat the procedure until we get to the end. average the perplexities, and use that to train the transformer end. 

I would further like this to be trained in a fair fashion, meaning that the transformer's learned normalizing flows should also have something like FairGNN (adversarial training against being able to determine the sample the tokens belong to).  